In [6]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [7]:
import core
import pandas as pd
import numpy as np

In [8]:
all_df = pd.read_csv("all.csv")
df_noreg = all_df[all_df["is_regularized"] == False]
df_noreg["is_big_task"] = False
df_noreg["dataset"] = df_noreg["dataset"].replace(
    {
        "cifar100_fixed": "cifar100",
        "imagenet100_subset_kaggle": "imagenet100",
    }
)

/var/folders/7p/77w4_2b565n0b8phw5c6dvtm0000gp/T/ipykernel_57596/3821391423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noreg["is_big_task"] = False
/var/folders/7p/77w4_2b565n0b8phw5c6dvtm0000gp/T/ipykernel_57596/3821391423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noreg["dataset"] = df_noreg["dataset"].replace(


In [23]:
finetuning_ewc_resnet, failed_runs, not_finished_runs = core.get_df_cl("01.20", 8)
len(failed_runs)
len(not_finished_runs)
finetuning_ewc_resnet.shape


finetuning_ewc_resnet = finetuning_ewc_resnet[
    (finetuning_ewc_resnet["network"].str.contains("resnet"))
    & (finetuning_ewc_resnet["approach"].isin(["finetuning", "ewc"]))
]

1

0

(119, 86)

In [24]:
finetuning_ewc_convnext, failed_runs, not_finished_runs = core.get_df_cl("01.20", 8)
len(failed_runs)
len(not_finished_runs)
finetuning_ewc_convnext.shape

finetuning_ewc_convnext = finetuning_ewc_convnext[
    (finetuning_ewc_convnext["network"].str.contains("convnext"))
    & (finetuning_ewc_convnext["approach"].isin(["finetuning", "ewc"]))
]

1

0

(119, 86)

In [11]:
big_all_or_lwf_replay, failed_runs, not_finished_runs = core.get_df_cl("01.28", 8)
len(failed_runs)
len(not_finished_runs)
big_all_or_lwf_replay.shape

big_all = big_all_or_lwf_replay[big_all_or_lwf_replay["is_big_task"]]
lwf_replay = big_all_or_lwf_replay[~big_all_or_lwf_replay["is_big_task"]]

2

15

(139, 86)

In [12]:
reg_fetril, failed_runs, not_finished_runs = core.get_df_pycil(
    "pycil-01.28-seed-fetril"
)
len(failed_runs)
reg_fetril.shape
noreg_fetril, failed_runs, not_finished_runs = core.get_df_pycil("pycil-01.27-new")
len(failed_runs)
noreg_fetril.shape
noreg_fetril = noreg_fetril[noreg_fetril["approach"] == "fetril"]

fetril = pd.concat([reg_fetril, noreg_fetril], axis=0)

big_task_fetril, failed_runs, not_finished_runs = core.get_df_pycil(
    "pycil-01.28-big_task"
)
len(failed_runs)
big_task_fetril.shape

0

(30, 38)

38

(54, 41)

0

(24, 38)

In [27]:
pivot_table = pd.pivot_table(
    finetuning_ewc_resnet[finetuning_ewc_resnet["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                   cifar100       imagenet100   
num_tasks                       5  10 20          10 20
approach   is_regularized                              
ewc        True                  0  0  0           0  0
finetuning True                  0  0  0           0  0

In [28]:
pivot_table = pd.pivot_table(
    finetuning_ewc_convnext[
        finetuning_ewc_convnext["network"].str.contains("convnext")
    ],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                   cifar100       imagenet100   
num_tasks                       5  10 20          10 20
approach   is_regularized                              
ewc        True                  0  0  0           0  0
finetuning True                  0  0  0           0  0

In [15]:
pivot_table = pd.pivot_table(
    big_all[big_all["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    big_all[big_all["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                   cifar100 imagenet100
num_tasks                        6           6
approach   is_regularized                     
ewc        False                 0           0
           True                  0           0
finetuning False                 0           0
           True                  0           0
lwf        False                 0           0
           True                  0           0
replay     False                 0           0
           True                  0           0

dataset                   cifar100 imagenet100
num_tasks                        6           6
approach   is_regularized                     
ewc        False               0.0         0.0
           True                0.0         0.0
finetuning False               0.0         0.0
           True                0.0         0.0
lwf        False               0.0         NaN
           True                0.0         NaN
replay     False               0.0         0.0
           True                0.0         0.0

In [16]:
pivot_table = pd.pivot_table(
    lwf_replay[lwf_replay["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    lwf_replay[lwf_replay["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                 cifar100       imagenet100   
num_tasks                     5  10 20          10 20
approach is_regularized                              
lwf      True                  0  0 -2           0  0
replay   True                  0  0  0           0  0

dataset                 cifar100           imagenet100
num_tasks                     5    10   20          10
approach is_regularized                               
lwf      True                0.0  0.0  0.0         NaN
replay   True                0.0  0.0  0.0         0.0

In [17]:
pivot_table = pd.pivot_table(
    df_noreg[df_noreg["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    df_noreg[df_noreg["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                   cifar100       imagenet_subset_kaggle   
num_tasks                       5  10 20                     10 20
approach   is_regularized                                         
ewc        False                 0  0  0                      0  0
finetuning False                 0  0  0                      0  0
lwf        False                 0  0  0                      0  0
replay     False                 0  0  0                      0  0

dataset                   cifar100       imagenet_subset_kaggle   
num_tasks                       5  10 20                     10 20
approach   is_regularized                                         
ewc        False                 0  0  0                      0  0
finetuning False                 0  0  0                      0  0
lwf        False                 0  0  0                      0  0
replay     False                 0  0  0                      0  0

In [18]:
pivot_table = pd.pivot_table(
    fetril[fetril["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    fetril[fetril["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                 cifar100       imagenet100   
num_tasks                     5  10 20          10 20
approach is_regularized                              
fetril   False                 0  0  0           0  0
         True                  0  0  0           0  0

dataset                 cifar100       imagenet100   
num_tasks                     5  10 20          10 20
approach is_regularized                              
fetril   False                 0  0  0           0  0
         True                  0  0  0           0  0

In [19]:
pivot_table = pd.pivot_table(
    big_task_fetril[big_task_fetril["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    big_task_fetril[big_task_fetril["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                 cifar100 imagenet100
num_tasks                     10          10
approach is_regularized                     
fetril   False                 0           0
         True                  0           0

dataset                 cifar100 imagenet100
num_tasks                     10          10
approach is_regularized                     
fetril   False                 0           0
         True                  0           0

In [29]:
allinone = pd.concat(
    [
        finetuning_ewc_resnet,
        finetuning_ewc_convnext,
        big_all,
        lwf_replay,
        fetril,
        big_task_fetril,
        df_noreg,
    ],
    axis=0,
)

/var/folders/7p/77w4_2b565n0b8phw5c6dvtm0000gp/T/ipykernel_57596/1657574346.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  allinone = pd.concat(


In [40]:
big_all = allinone[allinone["is_big_task"]]
no_big_all = allinone[~allinone["is_big_task"]]

In [214]:
resnet = pd.pivot_table(
    no_big_all[no_big_all["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
resnet
convnext = pd.pivot_table(
    no_big_all[no_big_all["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
convnext

dataset                      cifar100                         imagenet100  \
num_tasks                          5           10          20          10   
approach   is_regularized                                                   
ewc        False           23.10±0.51  12.22±0.11   4.91±2.37  13.73±0.16   
           True            27.21±0.34  11.35±3.13   6.04±2.99  21.09±0.54   
fetril     False           54.87±1.18  41.15±1.99  29.46±1.74  49.88±0.77   
           True            60.34±0.56  47.60±1.08  38.10±0.57  55.59±0.07   
finetuning False           20.55±0.35  11.54±0.10   5.03±0.61  11.52±0.12   
           True            23.62±0.21  11.97±1.82   6.39±2.79  14.93±0.19   
lwf        False           39.26±0.24  25.45±0.97  16.89±0.73  35.47±0.99   
           True            37.07±1.74  25.48±0.72  17.12±0.00  35.66±0.55   
replay     False           40.63±0.89  35.42±2.32  31.60±1.55  38.59±0.72   
           True            42.13±0.78  37.82±1.12  34.96±0.97  39.99±0.34   

dataset                                
num_tasks                          20  
approach   is_regularized              
ewc        False            7.01±0.12  
           True            11.83±0.06  
fetril     False           37.84±0.53  
           True            44.15±1.19  
finetuning False            6.08±0.24  
           True             9.21±0.47  
lwf        False           19.61±0.38  
           True            19.40±0.34  
replay     False           35.00±1.76  
           True            36.90±0.09

dataset                      cifar100                         imagenet100  \
num_tasks                          5           10          20          10   
approach   is_regularized                                                   
ewc        False           29.57±0.03  16.92±0.55   8.87±0.53  21.65±0.70   
           True            29.05±1.21  17.02±0.69  10.26±0.07  21.60±0.70   
fetril     False           39.93±1.82  31.75±1.86  27.74±0.95  34.79±1.97   
           True            46.85±0.61  37.99±1.90  32.69±0.66  39.31±0.65   
finetuning False           26.21±0.69  15.31±0.37   8.19±0.27  17.29±0.30   
           True            24.09±0.86  13.65±0.77   8.83±0.10  16.23±0.69   
lwf        False           40.70±0.09  29.83±0.21  23.01±0.87  43.73±0.46   
           True            44.52±0.66  31.38±1.02  21.58±0.38         NaN   
replay     False           34.53±0.25  28.59±0.13  25.42±0.25  34.55±0.86   
           True            34.73±0.85  27.91±0.63  26.06±0.55  34.13±0.79   

dataset                                
num_tasks                          20  
approach   is_regularized              
ewc        False           10.81±0.39  
           True            11.85±0.20  
fetril     False           23.62±1.69  
           True            31.18±1.68  
finetuning False            7.37±0.01  
           True             8.00±0.61  
lwf        False           29.60±0.71  
           True                   NaN  
replay     False           29.78±0.74  
           True                   NaN

In [52]:
pivot_table = pd.pivot_table(
    big_all[big_all["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table
pivot_table = pd.pivot_table(
    big_all[big_all["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
pivot_table

dataset                      cifar100 imagenet100
approach   is_regularized                        
ewc        False           22.94±0.60  21.03±0.66
           True            28.94±0.59  34.05±0.41
fetril     False           64.73±0.83  73.50±0.80
           True            67.60±0.51  74.84±0.94
finetuning False           19.54±0.58  18.94±0.31
           True            24.55±0.84  24.78±0.79
lwf        False           42.16±0.80  53.53±0.62
           True            42.67±0.45  54.19±0.25
replay     False           42.69±0.55  47.09±0.89
           True            44.95±0.28  49.58±0.65

dataset                      cifar100 imagenet100
approach   is_regularized                        
ewc        False           18.09±0.62  16.68±1.22
           True            39.03±0.88  51.32±0.92
fetril     False           42.48±1.57  33.97±7.95
           True            50.26±0.53  50.99±2.65
finetuning False           19.20±0.97  14.90±1.67
           True            31.09±1.20  38.59±1.20
lwf        False           44.58±0.65         NaN
           True            44.34±0.56         NaN
replay     False           36.21±0.74  42.98±0.48
           True            36.62±0.77  44.75±0.65

In [ ]:
# allinone["dataset"] = allinone["dataset"].replace(
#         {
#             "cifar100_fixed": "cifar100",
#             "imagenet_subset_kaggle": "imagenet100",
#         }
#     )

In [96]:
resnet = pd.pivot_table(
    no_big_all[no_big_all["network"].str.contains("resnet")],
    index=[
        "approach",
        "is_regularized",
    ],
    columns=["dataset", "num_tasks", "seed"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    # aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
resnet
convnext = pd.pivot_table(
    no_big_all[no_big_all["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)
convnext

dataset                   cifar100                                          \
num_tasks                       5                       10                   
seed                             0       1       2       0       1       2   
approach   is_regularized                                                    
ewc        False            23.690  22.440  23.160  12.190  12.100  12.370   
           True             27.140  26.830  27.660  15.700   8.470   9.870   
fetril     False            54.510  53.636  56.462  39.303  40.225  43.920   
           True             60.006  59.896  61.128  46.776  46.906  49.129   
finetuning False            20.880  20.690  20.070  11.690  11.450  11.490   
           True             23.400  23.550  23.910  14.350   9.920  11.630   
lwf        False            39.450  39.400  38.920  26.450  24.140  25.750   
           True             37.740  34.680  38.780  26.050  25.920  24.460   
replay     False            41.240  41.280  39.370  37.500  32.190  36.570   
           True             43.020  41.130  42.250  39.170  37.860  36.430   

dataset                                             imagenet100          \
num_tasks                       20                           10           
seed                             0       1        2           0       1   
approach   is_regularized                                                 
ewc        False            6.3200   1.570   6.8500      13.520  13.760   
           True             2.0500   9.240   6.8300      21.800  20.480   
fetril     False           31.6765  29.273  27.4285      49.392  49.283   
           True            38.5875  38.407  37.2985      55.688  55.558   
finetuning False            5.2500   4.200   5.6500      11.560  11.640   
           True             2.5200   9.010   7.6300      14.800  15.200   
lwf        False           17.6200     NaN  16.1600      34.180  36.600   
           True                NaN     NaN  17.1200      36.260  34.940   
replay     False           33.4300  29.650  31.7200      37.600  39.280   
           True            35.9700  33.660  35.2600      40.120  39.520   

dataset                                                      
num_tasks                               20                   
seed                            2        0       1        2  
approach   is_regularized                                    
ewc        False           13.920   6.9600   7.180   6.9000  
           True            20.980  11.8400  11.760  11.9000  
fetril     False           50.960  38.1445  37.104  38.2860  
           True            55.532  43.8415  42.877  45.7335  
finetuning False           11.360   6.3800   5.780   6.0800  
           True            14.780   9.8400   9.060   8.7200  
lwf        False           35.620  20.1400  19.420  19.2800  
           True            35.780  19.0000  19.840  19.3600  
replay     False           38.900  37.4600  34.060  33.4800  
           True            40.320  37.0200  36.800  36.8800

dataset                      cifar100                         imagenet100  \
num_tasks                          5           10          20          10   
approach   is_regularized                                                   
ewc        False           29.57±0.03  16.92±0.55   8.87±0.53  21.65±0.70   
           True            29.05±1.21  17.02±0.69  10.26±0.07  21.60±0.70   
fetril     False           39.93±1.82  31.75±1.86  27.74±0.95  34.79±1.97   
           True            46.85±0.61  37.99±1.90  32.69±0.66  39.31±0.65   
finetuning False           26.21±0.69  15.31±0.37   8.19±0.27  17.29±0.30   
           True            24.09±0.86  13.65±0.77   8.83±0.10  16.23±0.69   
lwf        False           40.70±0.09  29.83±0.21  23.01±0.87  43.73±0.46   
           True            44.52±0.66  31.38±1.02  21.58±0.38         NaN   
replay     False           34.53±0.25  28.59±0.13  25.42±0.25  34.55±0.86   
           True            34.73±0.85  27.91±0.63  26.06±0.55  34.13±0.79   

dataset                                
num_tasks                          20  
approach   is_regularized              
ewc        False           10.81±0.39  
           True            11.85±0.20  
fetril     False           23.62±1.69  
           True            31.18±1.68  
finetuning False            7.37±0.01  
           True             8.00±0.61  
lwf        False           29.60±0.71  
           True                   NaN  
replay     False           29.78±0.74  
           True                   NaN

In [290]:
def seed_wise_differcence(df):

    avg_acc = (
        df.groupby(["approach", "dataset", "num_tasks", "seed", "is_regularized"])[
            "avg_acc_tag"
        ]
        .agg(lambda x: x)
        .unstack()
    )
    avg_acc["difference"] = avg_acc[True] - avg_acc[False]
    avg_acc.reset_index(inplace=True)
    new_rows = avg_acc[
        ["approach", "dataset", "num_tasks", "seed", "difference"]
    ].copy()
    new_rows["is_regularized"] = "agg"  # Set is_regularized to 'add'
    new_rows["avg_acc_tag"] = new_rows[
        "difference"
    ]  # Set avg_acc_tag to the difference
    new_rows = new_rows[
        ["approach", "is_regularized", "dataset", "num_tasks", "avg_acc_tag", "seed"]
    ]
    df_final = pd.concat([df, new_rows], ignore_index=True)
    return df_final

In [297]:
big_all = allinone[allinone["is_big_task"]]
no_big_all = allinone[~allinone["is_big_task"]]

convnext_nobig = seed_wise_differcence(
    no_big_all[no_big_all["network"].str.contains("convnext")]
)
resnet_nobig = seed_wise_differcence(
    no_big_all[no_big_all["network"].str.contains("resnet")]
)
convnext_big = seed_wise_differcence(
    big_all[big_all["network"].str.contains("convnext")]
)
resnet_big = seed_wise_differcence(big_all[big_all["network"].str.contains("resnet")])

In [298]:
pd.pivot_table(
    resnet_nobig,
    index=[
        "approach",
        "is_regularized",
    ],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)

dataset                      cifar100                         imagenet100  \
num_tasks                          5           10          20          10   
approach   is_regularized                                                   
ewc        False           23.10±0.51  12.22±0.11   4.91±2.37  13.73±0.16   
           True            27.21±0.34  11.35±3.13   6.04±2.99  21.09±0.54   
           agg              4.11±0.47  -0.87±3.13   1.13±4.94   7.35±0.67   
fetril     False           54.87±1.18  41.15±1.99  29.46±1.74  49.88±0.77   
           True            60.34±0.56  47.60±1.08  38.10±0.57  55.59±0.07   
           agg              5.47±0.65   6.45±0.94   8.64±1.26   5.71±0.81   
finetuning False           20.55±0.35  11.54±0.10   5.03±0.61  11.52±0.12   
           True            23.62±0.21  11.97±1.82   6.39±2.79  14.93±0.19   
           agg              3.07±0.56   0.42±1.72   1.35±3.11   3.41±0.13   
lwf        False           39.26±0.24  25.45±0.97  16.89±0.73  35.47±0.99   
           True            37.07±1.74  25.48±0.72  17.12±0.00  35.66±0.55   
           agg             -2.19±1.90   0.03±1.29   0.96±0.00   0.19±1.53   
replay     False           40.63±0.89  35.42±2.32  31.60±1.55  38.59±0.72   
           True            42.13±0.78  37.82±1.12  34.96±0.97  39.99±0.34   
           agg              1.50±1.25   2.40±2.43   3.36±0.61   1.39±0.93   

dataset                                
num_tasks                          20  
approach   is_regularized              
ewc        False            7.01±0.12  
           True            11.83±0.06  
           agg              4.82±0.18  
fetril     False           37.84±0.53  
           True            44.15±1.19  
           agg              6.31±0.81  
finetuning False            6.08±0.24  
           True             9.21±0.47  
           agg              3.13±0.35  
lwf        False           19.61±0.38  
           True            19.40±0.34  
           agg             -0.21±0.67  
replay     False           35.00±1.76  
           True            36.90±0.09  
           agg              1.90±1.68

In [299]:
pd.pivot_table(
    convnext_nobig,
    index=[
        "approach",
        "is_regularized",
    ],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)

dataset                      cifar100                         imagenet100  \
num_tasks                          5           10          20          10   
approach   is_regularized                                                   
ewc        False           29.57±0.03  16.92±0.55   8.87±0.53  21.65±0.70   
           True            29.05±1.21  17.02±0.69  10.26±0.07  21.60±0.70   
           agg             -0.52±1.23   0.10±1.18   1.38±0.59  -0.05±0.56   
fetril     False           39.93±1.82  31.75±1.86  27.74±0.95  34.79±1.97   
           True            46.85±0.61  37.99±1.90  32.69±0.66  39.31±0.65   
           agg              6.91±1.59   6.25±0.52   4.94±1.60   4.52±1.40   
finetuning False           26.21±0.69  15.31±0.37   8.19±0.27  17.29±0.30   
           True            24.09±0.86  13.65±0.77   8.83±0.10  16.23±0.69   
           agg             -2.12±1.14  -1.66±0.57   0.64±0.33  -1.05±0.45   
lwf        False           40.70±0.09  29.83±0.21  23.01±0.87  43.73±0.46   
           True            44.52±0.66  31.38±1.02  21.58±0.38         NaN   
           agg              3.82±0.74   1.54±1.18  -1.43±0.82     nan±nan   
replay     False           34.53±0.25  28.59±0.13  25.42±0.25  34.55±0.86   
           True            34.73±0.85  27.91±0.63  26.06±0.55  34.13±0.79   
           agg              0.20±0.64  -0.68±0.60   0.64±0.81  -0.43±0.59   

dataset                                
num_tasks                          20  
approach   is_regularized              
ewc        False           10.81±0.39  
           True            11.85±0.20  
           agg              1.04±0.20  
fetril     False           23.62±1.69  
           True            31.18±1.68  
           agg              7.55±1.46  
finetuning False            7.37±0.01  
           True             8.00±0.61  
           agg              0.63±0.61  
lwf        False           29.60±0.71  
           True                   NaN  
           agg                nan±nan  
replay     False           29.78±0.74  
           True                   NaN  
           agg                nan±nan

In [302]:
pd.pivot_table(
    resnet_big,
    index=[
        "approach",
        "is_regularized",
    ],
    columns=["dataset"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)

dataset                      cifar100 imagenet100
approach   is_regularized                        
ewc        False           22.94±0.60  21.03±0.66
           True            28.94±0.59  34.05±0.41
           agg              6.01±0.87  13.02±1.01
fetril     False           64.73±0.83  73.50±0.80
           True            67.60±0.51  74.84±0.94
           agg              2.87±1.14   1.34±0.29
finetuning False           19.54±0.58  18.94±0.31
           True            24.55±0.84  24.78±0.79
           agg              5.01±0.92   5.84±0.64
lwf        False           42.16±0.80  53.53±0.62
           True            42.67±0.45  54.19±0.25
           agg              0.51±1.06   0.66±0.65
replay     False           42.69±0.55  47.09±0.89
           True            44.95±0.28  49.58±0.65
           agg              2.26±0.45   2.49±0.36

In [301]:
pd.pivot_table(
    convnext_big,
    index=[
        "approach",
        "is_regularized",
    ],
    columns=["dataset"],
    values="avg_acc_tag",
    # aggfunc=lambda x: len(x) - 3,
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    # fill_value=0,
)

dataset                      cifar100 imagenet100
approach   is_regularized                        
ewc        False           18.09±0.62  16.68±1.22
           True            39.03±0.88  51.32±0.92
           agg             20.94±1.46  34.64±1.96
fetril     False           42.48±1.57  33.97±7.95
           True            50.26±0.53  50.99±2.65
           agg              7.78±2.04  17.02±7.43
finetuning False           19.20±0.97  14.90±1.67
           True            31.09±1.20  38.59±1.20
           agg             11.89±0.40  23.68±2.56
lwf        False           44.58±0.65         NaN
           True            44.34±0.56         NaN
           agg             -0.24±0.18         NaN
replay     False           36.21±0.74  42.98±0.48
           True            36.62±0.77  44.75±0.65
           agg              0.41±0.40   1.77±0.77